In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import warnings
import datetime as dt
from scipy.interpolate import CubicSpline
from matplotlib import gridspec

import importlib
import sklearn

import xgboost as xgb

# local imports
from panda_helpers import *

from makedf import *

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 4.

dosave = False
savedir = "/icarus/app/users/gputnam/evtsel-ana/plots_Aug31/"

In [ ]:
fnu = "/icarus/data/users/gputnam/MCP2022GB/nu/nuall_old.df"

In [ ]:
nu_evtdf = pd.read_hdf(fnu, key="evtdf")

In [ ]:
nu_evtdf

In [ ]:
nu_hdr = pd.read_hdf(fnu, key="hdr")

In [ ]:
nu_hdr

In [ ]:
fic = "/icarus/data/users/gputnam/MCP2022GB/cosmic/intc_5000_old.df"

In [ ]:
ic_evtdf = pd.read_hdf(fic, key="evtdf")

In [ ]:
ic_evtdf

In [ ]:
ic_hdr = pd.read_hdf(fic, key="hdr")

In [ ]:
ic_hdr

In [ ]:
fdata = "/icarus/data/users/gputnam/MCP2022P/numi-commissioining-runs/prescaled_old.df"

In [ ]:
data_evtdf = pd.read_hdf(fdata, key="evtdf")

In [ ]:
data_evtdf

In [ ]:
data_hdr = pd.read_hdf(fdata, key="hdr")

In [ ]:
data_hdr

In [ ]:
def sample_concat(dfA, dfB, offset=1_000_000):
    dfB = dfB.copy()
    # dfB.index = dfB.index.set_levels(dfB.index.levels[0] + dfA.index.max()[0] + 1, level=0)
    dfB.index = dfB.index.set_levels(dfB.index.levels[0] + offset, level=0)
    return dfA.append(dfB)

In [ ]:
nu_pot = np.sum(nu_hdr.pot * nu_hdr.first_in_subrun)

In [ ]:
nu_evt_per_spill = np.sum(nu_hdr.ngenevt*nu_hdr.first_in_subrun) / (np.sum(nu_hdr.pot*nu_hdr.first_in_subrun) / 6e13)
ic_pot = np.sum(ic_hdr.ngenevt * ic_hdr.first_in_subrun * 6e13) / (1 - nu_evt_per_spill)

In [ ]:
data_pot = np.sum(data_hdr.pot)

In [ ]:
data_pot

In [ ]:
nu_evt_per_spill

In [ ]:
nu_evtdf["scale"] = data_pot / nu_pot
ic_evtdf["scale"] = ic_pot / nu_pot
data_evtdf["scale"] = 1

nu_evtdf["mc"] = True
ic_evtdf["mc"] = True
data_evtdf["mc"] = False

In [ ]:
evtdf = sample_concat(sample_concat(nu_evtdf, ic_evtdf), data_evtdf, 2_000_000)

In [ ]:
scale = evtdf.scale.copy()

In [ ]:
is_nu = (evtdf.slc.tmatch.idx >= 0) & evtdf.mc
is_nu.name = "$\\nu$"
is_cosmic =( evtdf.slc.tmatch.idx < 0) & evtdf.mc
is_cosmic.name = "Cosmic"

In [ ]:
categories = [is_cosmic, is_nu]

In [ ]:
def ratioplot(var, when, xlabel, bins=None, ylog=False, legendrow=False, areanorm=False):
    f, (axtop, axbot) = plt.subplots(2, gridspec_kw={'height_ratios': [2.5, 1]}, sharex=True, figsize=(6.4, 4.8*1.25))

    N,bins = np.histogram(var[when & ~evtdf.mc], weights=scale[when & ~evtdf.mc], bins=bins)
        
    areascale = 1. if not areanorm else np.sum(scale[when & ~evtdf.mc]) / np.sum(scale[when & evtdf.mc])
    
    Nmc,_,_ = axtop.hist([var[when & c] for c in categories],
                    weights=[areascale*scale[when & c] for c in categories],
                    bins=bins, 
                    label=[c.name for c in categories], 
                    stacked=True, linewidth=2)
    Nmc = Nmc[-1]
    centers =(bins[1:] + bins[:-1]) /2.
    
    # axtop.text(0.675, 0.275, "NuMI Data\nRuns 8437,\n8439, 8446\n%.2g POT" % data_pot, transform=axtop.transAxes)
    axtop.set_title("NuMI Runs 8437, 8439, 8446: %.2g POT" % data_pot)
    axtop.errorbar(centers, N, np.sqrt(N), linestyle="none",color="black", capsize=4,fmt='.', elinewidth=2, markersize='10', label="Data")

    axbot.set_xlabel(xlabel)
    if areanorm:
        axtop.set_ylabel("Area Normalized Entries")
    else:
        axtop.set_ylabel("POT Normalized Entries")
    
    legend_kwargs = {}
    if legendrow:
        legend_kwargs["ncol"] = len(categories) + 1
        
    axtop.legend(**legend_kwargs)
    
    if ylog:
        axtop.set_yscale("log")
    
    plt.subplots_adjust(wspace=0, hspace=0.02)

    axbot.errorbar(centers, N/Nmc, np.sqrt(N)/Nmc, linestyle="none",color="black", capsize=4,fmt='.', elinewidth=2, markersize='10', label="Data / MC")
    axbot.set_ylabel("Data / MC")
    # axbot.axhline([1], color="black", linestyle="--")
    
    axtop.grid()
    axbot.grid()

    return axbot, axtop

In [ ]:
var = evtdf.trunk.len
when = SlcInFV(evtdf.slc.vertex)
bins = np.linspace(0, 400, 21)

ratioplot(var, when, "Trunk Track Length [cm]", bins, True)

In [ ]:
var = evtdf.branch.len
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Branch Track Length [cm]", bins, True)

In [ ]:
var = evtdf.slc.vertex.x
bins = np.linspace(-300, 300, 31)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Slice Vertex X [cm]", bins)

In [ ]:
var = evtdf.slc.vertex.y
bins = np.linspace(-200, 200, 31)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Slice Vertex Y [cm]", bins)

In [ ]:
var = evtdf.slc.vertex.z
bins = np.linspace(-1e3, 1e3, 51)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Slice Vertex Z [cm]", bins, legendrow=True)

In [ ]:
var = evtdf.trunk.chi2pid.I2.chi2_proton
bins = np.linspace(0, 400, 21)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Trunk $\\chi^2_p$", bins)

In [ ]:
var = evtdf.trunk.chi2pid.I2.chi2_muon
bins = np.linspace(0, 80, 21)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Trunk $\\chi^2_\\mu$", bins)

In [ ]:
var = evtdf.trunk.chi2pid.I2.chi2_proton
bins = np.linspace(0, 400, 21)
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)

ratioplot(var, when, "Contained Trunk $\\chi^2_p$", bins)

In [ ]:
var = evtdf.trunk.chi2pid.I2.chi2_muon
bins = np.linspace(0, 80, 21)
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)

ratioplot(var, when, "Contained Trunk $\\chi^2_\\mu$", bins)

In [ ]:
var = evtdf.branch.chi2pid.I2.chi2_proton
bins = np.linspace(0, 400, 21)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Branch $\\chi^2_p$", bins)

In [ ]:
var = evtdf.branch.chi2pid.I2.chi2_muon
bins = np.linspace(0, 80, 21)
when = SlcInFV(evtdf.slc.vertex)

ratioplot(var, when, "Branch $\\chi^2_\\mu$", bins)

In [ ]:
var = evtdf.branch.chi2pid.I2.chi2_proton
bins = np.linspace(0, 400, 21)
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.end)

ratioplot(var, when, "Contained Branch $\\chi^2_p$", bins)

In [ ]:
var = evtdf.branch.chi2pid.I2.chi2_muon
bins = np.linspace(0, 80, 21)
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.end)

ratioplot(var, when, "Contained Branch $\\chi^2_\\mu$", bins)

In [ ]:
var = evtdf.slc.fmatch.time
when = SlcInFV(evtdf.slc.vertex) 
bins = np.linspace(-50, 50, 21)

axbot, axtop = ratioplot(var, when, "Flash Time [$\\mu$s]", bins=bins)

axbot.set_ylim([0,4.5])

In [ ]:
var = evtdf.slc.fmatch.score
when = SlcInFV(evtdf.slc.vertex) 
bins = np.linspace(0, 50, 21)

ratioplot(var, when, "Flash Score", bins=bins)

In [ ]:
evtdf[~evtdf.mc].slc.fmatch.time